## Reading text from the file

In [2]:
import nltk
import pandas as pd 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string 

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv('SMSSpamCollection.tsv', sep='\t')
data.columns = ['label', 'body_text']

def count_punch(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text)-text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punch%'] = data['body_text'].apply(lambda x: count_punch(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)

X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punch%'], pd.DataFrame(X_tfidf.toarray())], axis = 1)
X_features.head()

,body_len,punch%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Implementaion of grid-search 

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [10]:
X_train , x_test, Y_train, y_test = train_test_split(X_features, data['label'], test_size = 0.2)


In [14]:
def train_RF(n_est, depth):
    rf = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model = rf.fit(X_train, Y_train)
    y_pred = rf_model.predict(x_test)
    precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')
    print('Est: {} / Depth : {} ---------precision : {} / Accuracy : {}'. format(n_est, depth, round(precision, 3), round(recall, 3), round((y_pred == y_test).sum()/len(y_pred), 3)))

In [15]:
#Grid search 
for n_est in [10, 50, 100]:
    for depth in [10, 20, 30, None]:
        train_RF(n_est, depth)

Est: 10 / Depth : 10 ---------precision : 1.0 / Accuracy : 0.27
Est: 10 / Depth : 20 ---------precision : 0.976 / Accuracy : 0.582
Est: 10 / Depth : 30 ---------precision : 1.0 / Accuracy : 0.723
Est: 10 / Depth : None ---------precision : 0.991 / Accuracy : 0.766
Est: 50 / Depth : 10 ---------precision : 1.0 / Accuracy : 0.305
Est: 50 / Depth : 20 ---------precision : 1.0 / Accuracy : 0.617
Est: 50 / Depth : 30 ---------precision : 1.0 / Accuracy : 0.716
Est: 50 / Depth : None ---------precision : 1.0 / Accuracy : 0.801
Est: 100 / Depth : 10 ---------precision : 1.0 / Accuracy : 0.277
Est: 100 / Depth : 20 ---------precision : 1.0 / Accuracy : 0.631
Est: 100 / Depth : 30 ---------precision : 1.0 / Accuracy : 0.73
Est: 100 / Depth : None ---------precision : 1.0 / Accuracy : 0.809
